In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Preprocessing Data
* Detect Missing Dats
* Detect Categorial Variable
* Detect Outlier

In [ ]:
df_orig=pd.read_csv('/kaggle/input/clicks-conversion-tracking/KAG_conversion_data.csv')
df=df_orig.copy()

In [ ]:

def todo_list(df=df):
    print('To Do List:')
   
    null=df.isna().values.sum()
    cv=[]
    totaldata=[]
    #detect missing datas
    if null > 1:
        print("There " + "are" + str(null)+ " missing datas")
        if null>len(df)*0.05:
            print(" Suggest to replace datas")
            
        else:
            print(" Suggest to delete datas")
    elif null==1:
        print('There is 1 missing data')
        print('Suggest to fucking delete it you fucking dumb')
       
    else:
        print('There is no missing data asshole')
        
    
    #categorial variable
    for i in df:
        if df[i].dtypes==np.object:
            cv.append(i)
            totaldata.append(len(df[i].value_counts()))
    
    if len(cv) >1:
        print('There are '+ str(len(cv)) + " categorial variables")
        print(cv)
        print(totaldata)
    else:
        print('There is 1 categorial variable')
        print(cv)
        print(totaldata)
    

In [ ]:
todo_list()

In [ ]:
df

# ** Handling Catergorial Varible**

Instead of Dummies We Declare New Value

Gender
M=1
F=2

Age
30-34=1
35-39=2
40-44=3
45-49=4

In [ ]:
df['age']=df['age'].replace(['30-34','35-39','40-44','45-49'],[1,2,3,4])
df['gender']=df['gender'].replace(['M','F'],[1,2])
df['xyz_campaign_id']=df['xyz_campaign_id'].replace([916,936,1178],['A','B','C'])

In [ ]:
df

# Handling Outlier

In [ ]:
plt.figure(figsize=(20,20))
pd.DataFrame.boxplot(df.iloc[:,3:9])
plt.show()

In [ ]:
for i in df.iloc[:,3:9]:
    print(i)
    print(np.where(np.abs(stats.zscore(df[i])>3)))

In [ ]:
def outlier(column):
    q1=df[column].quantile(0.25)
    q3=df[column].quantile(0.75)
    IQR=q3-q1
    lower_range = q1- (1.5 * IQR)
    upper_range = q3 + (1.5 * IQR)
    return lower_range,upper_range


In [ ]:
target=['interest','Impressions','Clicks','Total_Conversion','Approved_Conversion']

In [ ]:
def bound(column):#bound-max and min
    q1=df[column].quantile(0.25)
    q3=df[column].quantile(0.75)
    IQR=q3-q1
    lower_range = q1- (1.5 * IQR)
    upper_range = q3 + (1.5 * IQR)
    return lower_range,upper_range


def outlier_detection(target):#inject list dari boundnya
    sumlower=[]
    sumupper=[]
    lower=[0]*len(target)
    upper=[0]*len(target)
    for index , i in enumerate(target):
        lower[index],upper[index]=bound(i)
        sumlower.append(lower[index])
        sumupper.append(upper[index])
    return sumlower,sumupper


def outlier_remover(target):
    sumlower,sumupper=outlier_detection(target)
    ndf=df.copy()
    for index, i in enumerate(target):
        ndf[i]=ndf[i][(ndf[i]>sumlower[index])&(ndf[i]<sumupper[index])]   
    return ndf

In [ ]:
df=outlier_remover(target)

In [ ]:
df

In [ ]:
for i in df.iloc[:,3:7]:
    print(i)
    print(np.where(np.abs(stats.zscore(df[i])>3)))

In [ ]:
df

plt.figure(figsize=(15,15))
plt.boxplot(df[''])

In [ ]:
df

In [ ]:
df=df.dropna()
df.isnull().sum()

In [ ]:
df

In [ ]:
plt.figure(figsize=(20,20))
pd.DataFrame.boxplot(df.iloc[:,5:11])
plt.show()

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr())

# Independent
* gender
* age
* spent

# Dependent 
* impression
* click
* total_conversion
* approved_conversion

In [ ]:
sns.lineplot(x='Spent',y='Clicks',data=df)

In [ ]:
A=df[df['xyz_campaign_id']=='A']
B=df[df['xyz_campaign_id']=='B']
C=df[df['xyz_campaign_id']=='C']
CTR_A = round(((A['Clicks'].sum()/A['Impressions'].sum())*100),3)
CTR_B = round(((B['Clicks'].sum()/B['Impressions'].sum())*100),3)
CTR_C = round(((C['Clicks'].sum()/C['Impressions'].sum())*100),3)
CTR_Total=[CTR_A,CTR_B,CTR_C]
x=['A','B','C']
print(CTR_Total)
plt.xlabel('Campaign')
plt.ylabel('Click Through Rate')
plt.title('CTR of 3 Campaigns')
plt.bar(x,CTR_Total)

In [ ]:
CR_A = round(((A['Approved_Conversion'].sum()/A['Clicks'].sum())*100),3)
CR_B = round(((B['Approved_Conversion'].sum()/B['Clicks'].sum())*100),3)
CR_C = round(((C['Approved_Conversion'].sum()/C['Clicks'].sum())*100),3)
CR_Total=[CR_A,CR_B,CR_C]
x=['A','B','C']
print(CR_Total)
plt.xlabel('Campaign')
plt.ylabel('Conversion Rate')
plt.title('CR of 3 Campaigns')
plt.bar(x,CR_Total)

In [ ]:
CC_A = round(((A['Spent'].sum()/A['Approved_Conversion'].sum())*100),3)
CC_B = round(((B['Spent'].sum()/B['Approved_Conversion'].sum())*100),3)
CC_C = round(((C['Spent'].sum()/C['Approved_Conversion'].sum())*100),3)
CC_Total=[CC_A,CC_B,CC_C]
x=['A','B','C']
print(CC_Total)
plt.xlabel('Campaign')
plt.ylabel('Conversion Cost')
plt.title('CC of 3 Campaigns')
plt.bar(x,CC_Total)

In [ ]:
data= {'CTR':CTR_Total,'CR':CR_Total,'CC':CC_Total,'Campaign':x}
dataframe=pd.DataFrame(data,columns=['CTR','CR','CC','Campaign'])
dataframe=dataframe.set_index('Campaign')
dataframe

In [ ]:
sns.barplot(y='Approved_Conversion',x='xyz_campaign_id',data=df)

In [ ]:
sns.barplot(y='Approved_Conversion',x='xyz_campaign_id',hue='gender',data=df)

In [ ]:
sns.barplot(y='Approved_Conversion',x='xyz_campaign_id',hue='age',data=df)

In [ ]:
sns.barplot(y='Impressions',x='age',hue='xyz_campaign_id',data=df)

In [ ]:
sns.barplot(y='Clicks',x='age',hue='xyz_campaign_id',data=df)

In [ ]:
sns.barplot(y='Total_Conversion',x='age',hue='xyz_campaign_id',data=ndf)

In [ ]:
sns.barplot(y='Approved_Conversion',x='age',hue='xyz_campaign_id',data=df)